In [1]:
"""Example usage:
  python object_detection/dataset_tools/create_oid_tf_record.py \
    --input_annotations_csv=/path/to/input/annotations-human-bbox.csv \
    --input_images_directory=/path/to/input/image_pixels_directory \
    --input_label_map=/path/to/input/labels_bbox_545.labelmap \
    --output_tf_record_path_prefix=/path/to/output/prefix.tfrecord
CSVs with bounding box annotations and image metadata (including the image URLs)
can be downloaded from the Open Images GitHub repository:
https://github.com/openimages/dataset
This script will include every image found in the input_images_directory in the
output TFRecord, even if the image has no corresponding bounding box annotations
in the input_annotations_csv.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import contextlib2
import pandas as pd
import tensorflow as tf

from object_detection.dataset_tools import oid_tfrecord_creation
from object_detection.utils import label_map_util

In [2]:
tf.flags.DEFINE_string('input_annotations_csv', None,
                       'Path to CSV containing image bounding box annotations')
tf.flags.DEFINE_string('input_images_directory', None,
                       'Directory containing the image pixels '
                       'downloaded from the OpenImages GitHub repository.')
tf.flags.DEFINE_string('input_label_map', None, 'Path to the label map proto')
tf.flags.DEFINE_string(
    'output_tf_record_path_prefix', None,
    'Path to the output TFRecord. The shard index and the number of shards '
    'will be appended for each output shard.')
tf.flags.DEFINE_integer('num_shards', 100, 'Number of TFRecord shards')

FLAGS = tf.flags.FLAGS

In [5]:
input_annotations_csv='/root/2017_07/validation/annotations-human-bbox.csv'
input_images_directory='/root/raw_images_validation'
input_label_map='../object_detection/data/oid_bbox_trainable_label_map.pbtxt'
!mkdir test_tfrecords
output_tf_record_path_prefix='test_tfrecords/test.tfrecord'
num_shards=100

mkdir: cannot create directory ‘test_tfrecords’: File exists


In [6]:
tf.logging.set_verbosity(tf.logging.INFO)

required_flags = [
  'input_annotations_csv', 'input_images_directory', 'input_label_map',
  'output_tf_record_path_prefix'
]

label_map = label_map_util.get_label_map_dict(input_label_map)
all_annotations = pd.read_csv(input_annotations_csv)
all_images = tf.gfile.Glob(
  os.path.join(input_images_directory, '*.jpg'))
all_image_ids = [os.path.splitext(os.path.basename(v))[0] for v in all_images]
all_image_ids = pd.DataFrame({'ImageID': all_image_ids})
all_annotations = pd.concat([all_annotations, all_image_ids])

tf.logging.log(tf.logging.INFO, 'Found %d images...', len(all_image_ids))

INFO:tensorflow:Found 41620 images...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [10]:
all_annotations = all_annotations.set_index("LabelName")

In [13]:
all_annotations=all_annotations.reset_index()

In [14]:
  with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = oid_tfrecord_creation.open_sharded_output_tfrecords(
        tf_record_close_stack, output_tf_record_path_prefix,
        num_shards)

    for counter, image_data in enumerate(all_annotations.groupby('ImageID')):
      tf.logging.log_every_n(tf.logging.INFO, 'Processed %d images...', 1000,
                             counter)

      image_id, image_annotations = image_data
      print(image_id)
      print("-----")
      print(image_annotations)
      print("-----")

        
#       break
      # In OID image file names are formed by appending ".jpg" to the image ID.
      image_path = os.path.join(input_images_directory, image_id + '.jpg')
      with tf.gfile.Open(image_path) as image_file:
        encoded_image = image_file.read()

      tf_example = oid_tfrecord_creation.tf_example_from_annotations_data_frame(
          image_annotations, label_map, encoded_image)
      if tf_example:
        shard_idx = int(image_id, 16) % num_shards
        output_tfrecords[shard_idx].write(tf_example.SerializeToString())
      break

INFO:tensorflow:Processed 0 images...
0001eeaf4aed83f9
-----
       LabelName  Confidence           ImageID  IsDepiction  IsGroupOf  \
61112   /m/0cmf2         1.0  0001eeaf4aed83f9          0.0        0.0   
206637       NaN         NaN  0001eeaf4aed83f9          NaN        NaN   

        IsInside  IsOccluded  IsTruncated Source      XMax      XMin  \
61112        0.0         0.0          0.0  human  0.964178  0.022464   
206637       NaN         NaN          NaN    NaN       NaN       NaN   

            YMax      YMin  
61112   0.800164  0.070656  
206637       NaN       NaN  
-----


In [15]:
image_annotations.LabelName.isin(label_map)

61112      True
206637    False
Name: LabelName, dtype: bool

In [16]:
image_annotations

,LabelName,Confidence,ImageID,IsDepiction,IsGroupOf,IsInside,IsOccluded,IsTruncated,Source,XMax,XMin,YMax,YMin
61112,/m/0cmf2,1.0,0001eeaf4aed83f9,0.0,0.0,0.0,0.0,0.0,human,0.964178,0.022464,0.800164,0.070656
206637,NaN,NaN,0001eeaf4aed83f9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
image_annotations[image_annotations.LabelName.isin(label_map)]

,LabelName,Confidence,ImageID,IsDepiction,IsGroupOf,IsInside,IsOccluded,IsTruncated,Source,XMax,XMin,YMax,YMin
61112,/m/0cmf2,1.0,0001eeaf4aed83f9,0.0,0.0,0.0,0.0,0.0,human,0.964178,0.022464,0.800164,0.070656
